# Title & Introduction

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import pyarrow
import networkx as nx
from networkx.algorithms import approximation as approx

In [2]:
import bq_helper
from bq_helper import BigQueryHelper

In [3]:
GOOGLE_APPLICATION_CREDENTIALS = r"C:\Users\Isaac\GA-Python\patents-isaac-ga-test-0795587125f7.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS 

In [4]:
from google.cloud import bigquery

## Fetch Data from BigQuery API
We need two separate tables for this analysis: patents and citations. Citations are retrieved via the Google BigQuery API using the BigQueryHelper package. 

In [5]:
#Configuration to retrieve citation data from BQ
bq_citations = bq_helper.BigQueryHelper(active_project="patents-isaac-ga-test", dataset_name="uspatentcitation")
def get_data(year):
    query = f"SELECT patent_id, citation_id, date FROM patents-isaac-ga-test.uspatentdata.uspatentcitation WHERE LEFT(date,4) = '{year}';"
    df = bq_citations.query_to_pandas(query)
    return df 

In [ ]:
#Retrieves data from BQ. Very long runtime, so should only be run once.
#Only configured to pull one year at at time to conserve memory. 
citations_raw = get_data(2010)

C:\ProgramData\Anaconda3\lib\site-packages\google\cloud\bigquery\client.py:461: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [ ]:
#Makes a copy of the citation data and removes design patents (non-numeric patent_ids) 
citations = citations_raw.copy()
mask_digit = ((citations['patent_id'].str.isdigit()) & (citations['citation_id'].str.isdigit()))
citations = citations[mask_digit]

Upload and clean raw patent class data


In [ ]:
#This cell actually retrieves the data from BigQuery. It takes between 5 and 10 minutes to run. 
bq_classes = bq_helper.BigQueryHelper(active_project="patents-public-data", dataset_name="patentsview")
query1 = f"SELECT patent_id, mainclass_id FROM patents-public-data.patentsview.uspc WHERE sequence = '0';"
patent_class_raw = bq_classes.query_to_pandas(query1)

In [ ]:
#makes a copy of the raw BQ data, drops all design patents and design citations as denoted by non-numeric patent_ids
patents_raw = patent_class_raw.copy()
patents_raw['mainclass_id'] = patents_raw['mainclass_id'].astype(str)
mask_digit2 = ((patents_raw['patent_id'].str.isdigit()) & (patents_raw['mainclass_id'].str.isdigit()))
patents_raw = patents_raw[mask_digit2]
patents_raw.head()

## Merging Class Data into Citations File

In [ ]:
#cast as string so we can use an inner join and drop everything that isn't in the list of utility patents. 
citations['patent_id'] = citations['patent_id'].astype(str)
citations['citation_id'] = citations['citation_id'].astype(str)

#merge in patent classes for citing patent
citations = pd.merge(citations, patents_raw, how= 'inner', left_on = 'patent_id', right_on= 'patent_id')
citations.rename(columns = {'mainclass_id':'patent_class'}, inplace = True)

In [ ]:
#merge in patent classes for citations
citations = pd.merge(citations, patents_raw, how= 'inner', left_on = 'citation_id', right_on= 'patent_id')
citations.rename(columns = {'mainclass_id':'citation_class', 'patent_id_x':'patent_id'}, inplace = True)
citations.drop(columns = ['patent_id_y'], inplace = True)

In [ ]:
citations.head()

## Get Node Weight

In [ ]:
#Get patent counts for use in the matrix later - size of bubble chart and calculation of edge weights
class_counts = pd.DataFrame(patents_raw.groupby(['mainclass_id'])['patent_id'].count())
class_counts.rename(columns = {'patent_id':'class_patent_count'}, inplace = True)
class_counts.head()

## Making the Map/Matrix
The core measure we're interested in is the share of citations for a given patent and patent class that come from another class. As this likely reflects the transfer of tacit knowledge for use in inventions, we can use this as a measure of how closely related the two classes are in a given time period. 

This approach does assume that all patent citations are equally important, which is manifestly false, but the impact of that assumption is ultimately very small. Subsequent analysis could use a patent's subsequent citations to retroactively determine how impactful it was. We could then adjust the patent citation share accordingly. However, that analysis requires a dramatically more powerful computer than I can currently afford, and the eventual matrix is not likely to be all that different. 

In [ ]:
citation_share = (1 / pd.DataFrame(citations.groupby(['patent_id'])['citation_id'].count()))
citation_share.rename(columns = {'citation_id':'citation_share'}, inplace = True)

In [ ]:
citations = pd.merge(citations, citation_share, how = 'left', left_on = 'patent_id', right_on = 'patent_id' )

In [ ]:
#Then we aggregate citationshare at the class level. 
#This measure tells us roughly how many patents in a class were derived from another class. 
matrix = pd.DataFrame(citations.groupby(['patent_class', 'citation_class'])['citation_share'].sum().reset_index())

In [ ]:
#merge class counts into the matrix. 
matrix = pd.merge(matrix, class_counts, how = 'inner', left_on = 'patent_class', right_on= 'mainclass_id')

In [ ]:
#dividing citation_share by the total class count tells us what fraction of the patents in a class derived from another class
matrix['edge_weight'] = (matrix['citation_share'] / matrix['class_patent_count'])

In [ ]:
matrix.head()

## Network Visualizations & Statistics

With networks of this size, a normal network diagram is all but unreadable, though when comparing charts, it's easier to view compare the overall shape of the knowledge space across multiple years. 

More importantly, the network diagram shown below is a byproduct of preparing the data for analysis. We've only selected one measure of network cohesion -- node closeness centrality, though the networkx package has an abundance of options. 

Please note that because all nodes are in some way connected to nearly all other nodes in this chart, most standard matrix measures are less useful that we might prefer, which is why we're not looking at degree centrality or node connectivity. 

In [ ]:
#strip leading zeros
matrix['patent_class'] = matrix['patent_class'].str.lstrip("0")
matrix['citation_class'] = matrix['citation_class'].str.lstrip("0")

In [ ]:
#edge_weight is already normalized between 0 and 1. we want larger values to be closer to zero or "closer" in the network
matrix['edge_weight']=  1 - matrix['edge_weight']

In [ ]:
#remove edges that have minimal value. 
matrix2 = matrix[matrix['edge_weight'] < 1]

In [ ]:
#create separate nodes and edges tables from matrix for easier graphing
nodes = matrix2[['patent_class', 'class_patent_count']].drop_duplicates()
edges = matrix2[['patent_class', 'citation_class', 'edge_weight']]

In [ ]:
edges = list(edges.to_records(index=False))
nodes = list(nodes.to_records(index= False))

In [ ]:
plt.figure(figsize = (30, 30))
g = nx.Graph()

for node, size in nodes:
    g.add_node(node, size = size)

for start, end, length in edges:
    g.add_edge(start, end, length = length)

nx.draw_networkx(g, arrows = True, with_labels = False, alpha = .5, node_size = 10)
plt.savefig('2000Patents.png', dpi=300, bbox_inches='tight')

class_closeness_centrality = nx.algorithms.centrality.closeness_centrality(g, distance= length)

In [ ]:
close_nodes = pd.DataFrame(list(class_closeness_centrality.items()))
close_nodes.rename(columns= {0:"class", 1:'closeness'}, inplace= True)
close_nodes.sort_values('closeness', ascending = False, inplace= True)
close_nodes

## Output
Writes the total patent citation matrix and the node closeness centrality scores to JSON files for use elsewhere. 
Note that the graph visualization was exported previously to .png. 

In [ ]:
matrix.to_json('C:\Users\Isaac\GA-Python\2000matrix.json')
close_nodes.to_json('C:\Users\Isaac\GA-Python\2000closeness.json')

## Citations
All data drawn from the PatentView database, which is supplied and updated by the USPTO and hosted by Google in the BigQuery Public Data project. 

Methods drawn from Rigby, Kogler & Tucker (2013) - Mapping Technological Relatedness in US Cities

The Networkx graphing and analysis package is fantastic, powerful, and very poorly documented. 